In [ ]:
!pip install -q transformers accelerate bitsandbytes peft trl
# torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers
!pip install --upgrade trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)

In [ ]:
model_name = "meta-llama/Meta-Llama-3.1-8B"

In [ ]:
"""**HF account Configuration**"""
config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]

In [ ]:
"""**Quantization Configuration**"""
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
"""**Loading the Tokenizer and the LLM**"""
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128
)

In [ ]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:
prompt = "What is stroke in medical terms?"

In [ ]:
llama3_response = get_response(prompt)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [ ]:
llama3_response

'What is stroke in medical terms? A stroke is a life-threatening medical emergency that requires immediate attention and treatment. It occurs when blood flow to part of the brain is blocked or interrupted, causing brain cells to die. The effects of a stroke can vary depending on the severity of the damage to the brain, but they can range from mild to severe and can include paralysis, speech problems, and memory loss.\nWhat is a stroke in medical terms?\nA stroke is a life-threatening medical emergency that requires immediate attention and treatment. It occurs when blood flow to part of the brain is blocked or interrupted, causing brain cells to die. The effects of a stroke can vary depending on the'

In [ ]:
print(llama3_response[len(prompt):])

 A stroke is a life-threatening medical emergency that requires immediate attention and treatment. It occurs when blood flow to part of the brain is blocked or interrupted, causing brain cells to die. The effects of a stroke can vary depending on the severity of the damage to the brain, but they can range from mild to severe and can include paralysis, speech problems, and memory loss.
What is a stroke in medical terms?
A stroke is a life-threatening medical emergency that requires immediate attention and treatment. It occurs when blood flow to part of the brain is blocked or interrupted, causing brain cells to die. The effects of a stroke can vary depending on the


In [ ]:
from datasets import load_dataset

dataset = load_dataset("health_fact")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


health_fact.py:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.61k [00:00<?, ?B/s]

The repository for health_fact contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/health_fact.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1235 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1225 [00:00<?, ? examples/s]

In [ ]:

from transformers import Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType

#the tokenization function to tokenize both claim and explanation fields
def tokenize_function(examples):
    input_texts = examples['claim']
    target_texts = examples['explanation']
    inputs = tokenizer(input_texts, padding="max_length", truncation=True)
    targets = tokenizer(target_texts, padding="max_length", truncation=True)

    # Combine inputs and targets into a single dictionary
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

In [ ]:
# Implement LoRA technique and fine-tune the model
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Train the model
trainer.train()

In [ ]:
#Save the model and tokenizer
model.save_pretrained("./fine-tuned-llama")
tokenizer.save_pretrained("./fine-tuned-llama")

In [ ]:
#Load the fine-tuned model and tokenizer
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./fine-tuned-llama")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-llama")

In [ ]:
#Generate a response with the fine-tuned model
input_text = "Tell me something about diabetes."
inputs = fine_tuned_tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = fine_tuned_model.generate(**inputs)
response = fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)